In [2]:
import requests
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
data = pd.read_csv("../data/old/yappy_hackaton_2024_400k.csv").iloc[40001:40005]
data.fillna("", inplace=True)

In [4]:
data.T.to_dict()

{40001: {'link': 'https://cdn-st.rutubelist.ru/media/0f/fc/00fc6050405ca29a97843d91b4ef/fhd.mp4',
  'description': 'Нравится ее стиль'},
 40002: {'link': 'https://cdn-st.rutubelist.ru/media/8a/04/c51a9e644f578ac328c778124d02/fhd.mp4',
  'description': 'Стесняшка Жэньши • Монолог фармацевта | Kusuriya no Hitorigoto'},
 40003: {'link': 'https://cdn-st.rutubelist.ru/media/9f/27/4edb34bc4ce182bfcd9c6822bd56/fhd.mp4',
  'description': '#спорт #наспорте #растяжка #фитнес #мотивация'},
 40004: {'link': 'https://cdn-st.rutubelist.ru/media/13/83/ed72c0e847c9964bd75061297e5a/fhd.mp4',
  'description': '#красивыедевушки #губы #макияж'}}

In [26]:
%%time

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

json_data = data.iloc[1].to_dict()
response = requests.post('http://localhost:8080/index', headers=headers, json=json_data)

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 7.15 s


In [27]:
%%time

headers = {
    'accept': 'application/json',
}

params = {
    'text': 'даня милохин',
}

response = requests.get('http://localhost:8080/search', params=params, headers=headers)

CPU times: user 2.3 ms, sys: 0 ns, total: 2.3 ms
Wall time: 41 ms


In [28]:
responses = []

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

for i in tqdm(range(500)):
    json_data = data.iloc[i].to_dict()
    response = requests.post('http://localhost:8080/index', headers=headers, json=json_data)
    if response.status_code != 200:
        print(i, response.text)
    responses.append(response)

In [29]:
import aiohttp, asyncio

async def post_request(session, url, headers, json_data):
    async with session.post(url, headers=headers, json=json_data) as response:
        if response.status != 200:
            print(json_data, await response.text())
        return await response.json()

responses = []
batch_size = 5
url = 'http://localhost:8080/index'

async with aiohttp.ClientSession() as session:
    for i in tqdm(range(0, len(data), batch_size)):
        tasks = []
        for j in range(i, min(i + batch_size, len(data))):
            json_data = data.iloc[j].to_dict()
            tasks.append(post_request(session, url, headers, json_data))
        responses.extend(await asyncio.gather(*tasks))